# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.7,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = city_data_df[(city_data_df['Max Temp']<26.67) & (city_data_df['Max Temp']>23.80)]

# Drop any rows with null values
ideal_temp = ideal_temp.dropna()

# Display sample data
ideal_temp

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,labuhan,-6.8844,112.2051,25.24,88,100,2.74,ID,1666108254
46,46,mogadishu,2.0371,45.3438,25.67,83,13,6.45,SO,1666108258
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
52,52,businga,3.3333,20.8833,24.82,87,99,0.97,CD,1666108261
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
...,...,...,...,...,...,...,...,...,...,...
562,562,tena,-0.9833,-77.8167,25.09,62,100,1.58,EC,1666108557
566,566,nhulunbuy,-12.2333,136.7667,26.34,78,88,0.00,AU,1666108559
567,567,sangamner,19.5667,74.2167,23.82,85,100,2.28,IN,1666108559
569,569,ranong,9.9658,98.6348,24.34,93,100,1.44,TH,1666108330


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ['City','Country','Lat','Lng','Humidity']

hotel_df = city_data_df[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,
1,farsund,NO,58.0948,6.8047,100,
2,new norfolk,AU,-42.7826,147.0587,58,
3,jamestown,US,42.0970,-79.2353,77,
4,lanzhou,CN,36.0564,103.7922,48,
...,...,...,...,...,...,...
572,sovetskiy,RU,61.3614,63.5842,96,
573,tatarsk,RU,55.2190,75.9828,89,
574,canchungo,GW,12.0672,-16.0333,61,
575,iquique,CL,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Sømandshjemmet
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hot

cacahuatepec - nearest hotel: No hotel found
paita - nearest hotel: Victor's
bumba - nearest hotel: Hotel Bumba
maragogi - nearest hotel: Pousada Mariluz
kosa - nearest hotel: Гостиница "Рады"
lagoa - nearest hotel: Casa da Madrinha
cabra - nearest hotel: Pensión Guerrero
gat - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
asau - nearest hotel: Hotel Romanița
isangel - nearest hotel: Tanna Lodge
pevek - nearest hotel: No hotel found
husavik - nearest hotel: Fosshótel Húsavík
kodiak - nearest hotel: Shelikof Lodge
zyryanka - nearest hotel: No hotel found
valparaiso - nearest hotel: Ibis Hotel
baykit - nearest hotel: No hotel found
port moresby - nearest hotel: The Sanctuary Hotel and Spa
meulaboh - nearest hotel: Hotel Meuligou
nakamura - nearest hotel: ホテル アイランド
simao - nearest hotel: Marvelous Land Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
bethel - nearest hotel: Hampton Inn Danbury
kodinsk - nearest hotel: Ангара
luderitz - nearest hotel: Seavie

mayumba - nearest hotel: Résidence Hôtelière Fabso
adrar - nearest hotel: No hotel found
port blair - nearest hotel: Andaman Ocean Breeze
komsomolskiy - nearest hotel: Диёр (Диор)
ribeira grande - nearest hotel: Hospedaria JSF
qasigiannguit - nearest hotel: Hotel Diskobay
buraydah - nearest hotel: دنا للوحدات السكنية
port-gentil - nearest hotel: Tara-Mé
saint-francois - nearest hotel: Chez Lily
vestmannaeyjar - nearest hotel: Westman Islands Inn
bogande - nearest hotel: No hotel found


KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    hover_cols=['Hotel Name', 'Country'],
    scale = 0.7,
    color = "City"
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)